In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('./crawler.xlsx')

df

,Url,OE,Vehicle,Pic,Src,kompressoröl:,herstellereinschränkung:,öl-füllmenge [ml]:,kompressor-id:,riemenscheiben-ø [mm]:,...,AUTO AIR GLOUCESTER,DENSO,HC-PARTS,LIZARTE,MOTRIO,QUINTON HAZELL,TEAMEC,VALEO,WAECO,EACLIMA
0,https://www.autoteiledirekt.de/nissens-7275624...,21000205;7H0820805D,"VW Multivan T5 (7HM, 7HN, 7HF, 7EF, 7EM, 7EN) ...",NaN,https://cdn.autoteiledirekt.de/thumb?id=727562...,PAG 100,SANDEN,180,SD7H15,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoteiledirekt.de/nissens-7850717...,000771974;71721705;71721706,FIAT Cinquecento (170) (Baujahr 07.1991 - 07.1...,NaN,https://cdn.autoteiledirekt.de/thumb?id=785071...,PAG 46,SANDEN,100,SD7B10,115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoteiledirekt.de/nissens-7275626...,21000202;1135025;1135295;1135323;1854032;18540...,OPEL Combo B Kastenwagen (Baujahr 07.1994 - 10...,NaN,https://cdn.autoteiledirekt.de/thumb?id=727562...,PAG 46,DELPHI,150,V5,125,...,140223.0,DCP20009,AC298,810601009.0,8.671006e+09,"662310, 1163204709",8600047.0,"699072, 187081, 197059",8.880100e+09,NaN
3,https://www.autoteiledirekt.de/nissens-7023595...,21000094;7M0820803M;7M0820803Q;98NW-19D629-AB,"SEAT Alhambra I (7V8, 7V9) (Baujahr 04.1996 - ...",NaN,https://cdn.autoteiledirekt.de/thumb?id=702359...,PAG 46,SANDEN,135,SD7V16,119,...,141164.0,DCP32013,NaN,NaN,NaN,NaN,8646003.0,699305,NaN,20A1162


In [3]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import time

# = = = = = = = = = = = = = = = = = =

from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = = = = =

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column+1):
    if ws.cell(1, column).value == 'Src':
        index_src = column
    elif ws.cell(1, column).value == 'Pic':
        index_pic = column
        
print('Src索引：' + str(index_src))
print('Pic索引：' + str(index_pic))
print()

# = = = = = = = = = = = = = = = = = =

work = Queue()
for row in range(2, ws.max_row+1):
    work.put_nowait(row)

# = = = = = = = = = = = = = = = = = =

def crawler():
    global ws
    
    while not work.empty():
        row = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        src = ws.cell(row, index_src).value.replace('https://', '').replace('http://', '').replace('/', '_').replace('?', '_') + '.jpeg'
        
        status = 'error'
        for _ in range(11):
            if src == None or src == '':
                break
                
            try:
                pic = Image('./pic/' + src)
                
                # = = = = = = = = = = = = = = = = = =
                
                if pic.width >= pic.height:
                    pic.height = 50.0 / pic.width * pic.height
                    pic.width = 50.0
                else:
                    pic.width = 50.0 / pic.height * pic.width
                    pic.height = 50.0
                    
                # = = = = = = = = = = = = = = = = = =

                ws.add_image(pic, get_column_letter(index_pic) + str(row))
                
                status = 'ok'
            
                break
            
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = = = = =
        
        if src != None and src != '':
            print(src +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('输出中...')
wb.save('./crawler-pic.xlsx')
print()
print('搞定！')

Src索引：5
Pic索引：4

cdn.autoteiledirekt.de_thumb_id=7275624&m=1&n=0&lng=de&rev=94077844.jpeg  <->  [ok] - 剩余数量：3
cdn.autoteiledirekt.de_thumb_id=7850717&m=1&n=0&lng=de&rev=94077844.jpeg  <->  [ok] - 剩余数量：2
cdn.autoteiledirekt.de_thumb_id=7275626&m=1&n=0&lng=de&rev=94077844.jpeg  <->  [ok] - 剩余数量：1
cdn.autoteiledirekt.de_thumb_id=7023595&m=1&n=0&lng=de&rev=94077844.jpeg  <->  [ok] - 剩余数量：0

输出中...

搞定！
